In [64]:
import numpy as np
import time
from collections import defaultdict

In [66]:
def cube_vol(b):
    x1, x2 = b[0]
    y1, y2 = b[1]
    z1, z2 = b[2]
    return (abs(x2 - x1) + 1) * (abs(y2 - y1) + 1) * (abs(z2 - z1) + 1)

In [54]:
def get_data(fname):
    arr = []
    with open(fname) as f:
        lines = f.readlines() 
        for l in lines:
            status, line = l.rstrip().split(' ')
            if status == 'on':
                status = 1
            else:
                status = 0
            line = line.split(',')
            arr.append((status, (tuple(list(map(int, line[0][2:].split('..')))),
                                       tuple(list(map(int, line[1][2:].split('..')))),
                                       tuple(list(map(int, line[2][2:].split('..')))))))
    return arr

In [28]:
def get_min_max(dat):
    min_x = int(dat[0][1][0])
    max_x = int(dat[0][1][1])
    min_y = int(dat[0][2][0])
    max_y = int(dat[0][2][1])
    min_z = int(dat[0][3][0])
    max_z = int(dat[0][3][1])
    for d in dat[1:]:
        x1 = int(d[1][0])
        x2 = int(d[1][1])
        y1 = int(d[2][0])
        y2 = int(d[2][1])
        z1 = int(d[3][0])
        z2 = int(d[3][1])
        if x1 < min_x:
            min_x = x1
        if x2 > max_x:
            max_x = x2
        if y1 < min_y:
            min_y = y1
        if y2 > max_y:
            max_y = y2
        if z1 < min_z:
            min_z = z1
        if z2 > max_z:
            max_z = z2
    return min_x, max_x, min_y, max_y, min_z, max_z

In [7]:
data = get_data('input.txt')

In [30]:
min_x, max_x, min_y, max_y, min_z, max_z = get_min_max(data)

In [42]:
print(min_x)
print(max_x + abs(min_x))
print(min_y)
print(max_y + abs(min_y))
print(min_z)
print(max_z + abs(min_y))

-98592
196691
-93976
190409
-97578
191754


In [70]:
data = get_data('input.txt')
cuboids = np.zeros((101, 101, 101))
t0 = time.time()
for ind, d in enumerate(data):
    status, bounds = d
    x1, x2 = bounds[0]
    y1, y2 = bounds[1]
    z1, z2 = bounds[2]
    
    x1 += 50
    x2 += 50
    y1 += 50
    y2 += 50
    z1 += 50
    z2 += 50

    if x1 > 100:
        continue
        
    if y1 > 100:
        continue
        
    if z1 > 100:
        continue
    
    if x1 <= 0:
        x1 = 0
        if x2 < 0:
            continue
        if x2 > 100:
            x2 = 100
        
    if y1 <= 0:
        y1 = 0
        if y2 < 0:
            continue
        if y2 > 100:
            y2 = 100
        
    if z1 <= 0 and z2 >= 0:
        z1 = 0
        if z2 < 0:
            continue
        if z2 > 100:
            z2 = 100
    cuboids[x1:x2+1,y1:y2+1, z1:z2+1] = d[0]
t1 = time.time()
total = t1-t0
print(f'time: {total}')
len(np.where(cuboids == 1)[0])

time: 0.004015445709228516


610196

In [62]:
def overlaps(b1, b2):
    ans = []
    for n1, n2 in zip(b1, b2):
        if n1[1] < n2[0] or n2[1] < n1[0]:
            return None
        bounds = (max(n1[0], n2[0]), min(n1[1], n2[1]))
        ans.append(bounds)
    return tuple(ans)

In [61]:
def count(steps):
    counts = defaultdict(int)
    for s in steps:
        switch, bounds = s
        updates = defaultdict(int)
        keys = set(counts.keys())
        for cube in keys:
            overlapping = overlaps(bounds, cube)
            if not overlapping:
                continue
            updates[overlapping] -= counts[cube]
        if switch == 1:
            updates[bounds] += 1
        for c in updates:
            counts[c] += updates[c]
    return counts

In [57]:
data = get_data('input.txt')

In [71]:
counts = count(data)
p2 = 0
for cube in counts:
    p2 += cube_vol(cube) * counts[cube]
print(f"Part 2: {p2}")

Part 2: 1282401587270826
